In [45]:
import pandas as pd
from datetime import datetime, timedelta

In [46]:
from getNewsSoup import get_news
from getNewsSoup import get_content
from getNewsSoup import get_news_from_finans
from getNewsSoup import get_news_from_roza

In [10]:
path = '../Data/news-data.csv'

past_dates = [
    '/20' + date.split('-')[0].split('.')[::-1][0] + 
        '/' +date.split('-')[0].split('.')[::-1][1] + 
            '/' + date.split('-')[0].split('.')[::-1][2]
                for date in pd.read_csv(path)['Day_month_year_hour']
        ]

print("total news : ", len(past_dates))
past_dates = list(set(past_dates))
print("total days :", len(past_dates))

total news :  180462
total days : 2670


In [48]:
max_day = 3000
start_day = 0
end_day = start_day + max_day

In [49]:
# Hesaplanan tarihlerin listesi
dates = []

def main(start_day, end_day, getToday):
    # Şu anki tarih ve saat bilgisini al
    current_date = datetime.now()
    
    if(getToday):
        # Şu anki yıl, ay ve gün bilgilerini al
        year = str(current_date.year)
        month = str(current_date.month)
        day = str(current_date.day)

        # Ay ve gün bilgilerini iki haneli yapma (01, 02, ..., 09)
        if len(month) == 1:
            month = "0" + month
        if len(day) == 1:
            day = "0" + day

        # Bugünün tarihini "/YYYY/MM/DD" formatında listeye ekleme
        dates.append("/" + year + "/" + month + "/" + day)
    
    # Belirtilen aralıkta tarihleri oluşturma
    for i in range(start_day, end_day):
        # Güncel tarihten önceki günleri hesapla
        previous_date = current_date - timedelta(days= i)

        # Önceki tarihin yıl, ay ve gün bilgilerini al
        year = str(previous_date.year)
        month = str(previous_date.month)
        day = str(previous_date.day)

        # Ay ve gün bilgilerini iki haneli yapma (01, 02, ..., 09)
        if len(month) == 1:
            month = "0" + month
        if len(day) == 1:
            day = "0" + day

        # Oluşturulan tarihi "/YYYY/MM/DD" formatında listeye ekleme
        date = "/" + year + "/" + month + "/" + day

        if date not in past_dates:
            dates.append(date)

    # Oluşturulan tarih listesini geri döndürme
    return dates

# Fonksiyonu çağırarak belirli bir gün aralığındaki tarihleri hesapla
dates = list(set(main(start_day=start_day, end_day=end_day, getToday=False)))
print(f"len Dates list {len(dates)}")

len Dates list 552


In [51]:
content_urls = []  
# Haber türlerini (örneğin, yaşam, ekonomi, spor) tutacak liste
news_types = []  
# Haber başlıklarını tutacak liste
titles = []  
# Haber tarih ve saat bilgilerini tutacak liste
date_time_list = []  
# Haber görsel URL'lerini tutacak liste
image_urls = []  
# Haber içeriklerini tutacak liste
contents = []  

df = pd.DataFrame(columns=['Title', 'Content', 'Content_url', 'News_type', 'Day_month_year_hour', 'Img_url'])

def append_data(title="", content='', content_url='', news_type='', date_time='', img_url=''):
    df.loc[len(df)] = [title, content, content_url, news_type, date_time, img_url]

In [31]:
# Geçerli URL'nin indeksi
current_url_index = 0  

# Yasaklı URL'ler listesi, farklı HTML yapılarına sahip olanlar
banned_urls = []
# Yasaklı haberlerin listesi
banned_news = []
# Haber içeriklerinin URL'lerini tutacak liste

# Yasaklı bağlantıların listesi
banned_hrefs = []

# Açilmayan sayfa sayısı
counter = 0

# Ana döngü, belirli bir tarih aralığındaki haberleri almak için kullanılır
while True:
    # Veriyi almak için URL'yi fonksiyona veriyoruz.
    soup = get_news(url="/timeline" + dates[current_url_index], day_news=True)
    
    # Sayfa açılmadıysa bu if çalışmaz ve current_url_index değişmez, böylece sayfa değişmez
    if soup is not None and len(soup.find_all('div', attrs={'class': 'box'})) != 0:
        
        # İlerlemeyi adım adım takip etmek için
        remaining_days = len(dates) - (current_url_index )
        if remaining_days % 10 == 0:
            print(f"{remaining_days} days left")
        
        if remaining_days % 5:
            df.to_csv("../Data/data_39.csv", encoding='utf-8', index=False)
            
        # news_boxes, o günün bütün haberlerini içeriyor
        for box in soup.find_all('div', attrs={'class': 'box'}):
            box_classes = box.get("class")

            # Her bir haberin class'i 'box['box'], reklam olanların class'i 'box advert'['box', 'advert'], uzunluğu 2 olduğu için reklamları almıyoruz
            if len(box_classes) == 2:
                pass

            # Eğer class sadece tek bir değerden oluşuyorsa ('box') o zaman bu haberi alıyoruz
            else:
                # Haber içeriğinin URL'sini al
                content_url = box.find('a').get("href")
                
                try:
                    head_url = content_url.split("/")[1]        

                    # Haberin türünü al
                    news_type = box.find_all("div", attrs={"class": "info"})[0].find("a").text.strip()
                    
                    # Haber başlığını al
                    title = box.find('strong').text.strip() 
                    
                    # Haber tarih ve saat bilgisini al
                    current_date = box.find('div', {'class': "info"}).find("label").text.strip()
                    date_time = current_date.split(" ")[0].split(",")[0] + '-' + current_date.split(" ")[-1]
                    
                    # Haber görsel URL'sini al
                    image_url = box.find("img").get('src')

                    # URL 'roza' ile başlıyorsa, özel bir fonksiyon kullanarak haber içeriğini al
                    if head_url == "roza":
                        roza_news = get_news_from_roza(content_url)
                        append_data(title=title, content=roza_news, content_url=content_url, news_type=news_type, date_time=date_time, img_url=image_url)

                    # 'roza' ile başlamayan ve yasaklı URL'ler içinde olmayan haber başlıklarını al
                    elif head_url not in banned_urls:
                        content_soup = get_news(url=content_url)
                        if content_soup is not None:
                            
                            content_check = content_soup.find_all("div", attrs={"class": "col-sm-12 view20 pagedItems"})

                            # Eğer haber içeriğinde birden fazla fotoğraf varsa class yapısı değişiyor ve burada o haberin bu sayfa yapısına sahip olup 
                            # olmadığını kontrol ediyoruz 
                            if len(content_check) == 0:
                                news_content = get_content(content_soup=content_soup)
                                append_data(title=title, content=news_content, content_url=content_url, news_type=news_type, date_time=date_time, img_url=image_url)
                            # Eğer haber birden fazla fotoğraf ve içerikten oluşuyorsa bu şart sağlanır
                            else:    
                                news_content = get_content(content_soup=content_soup, url=" pagedItems")
                                append_data(title=title, content=news_content, content_url=content_url, news_type=news_type, date_time=date_time, img_url=image_url)
                    
                    elif head_url == "finans":
                        
                        finans_news = get_news_from_finans(content_url)
                        append_data(title=title, content=finans_news, content_url=content_url, news_type=news_type, date_time=date_time, img_url=image_url)
                        
                    else:
                        # Eğer URL 'roza', 'finans' ve 'spor' sayfalarına giriyorsa geç
                        banned_hrefs.append(content_url)
                except:
                    pass
        current_url_index += 1        

    # Eğer sayfa açılmazsa current_url_index değişmez ve döngü tekrar aynı sayfayı deneyecektir
    else:
        counter += 1

    # İstenen gün sayısına ulaşıldığında döngüyü sonlandır
    if current_url_index == len(dates):
        break

340 days left
330 days left
320 days left
310 days left
300 days left


IndexError: list index out of range

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17771 entries, 0 to 17770
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Title                17771 non-null  object
 1   Content              17771 non-null  object
 2   Content_url          17771 non-null  object
 3   News_type            17771 non-null  object
 4   Day_month_year_hour  17771 non-null  object
 5   Img_url              17771 non-null  object
dtypes: object(6)
memory usage: 971.9+ KB


In [33]:
df.to_csv("../Data/data_39.csv", encoding='utf-8', index=False)